[//]: #![erdos-logo](risecamp/erdos-pylot-risecamp-logo.png)
<img src="risecamp/erdos-pylot-risecamp-logo.png" alt="erdos-logo" style="width: 800px;"/>

<font color="red"><b>TODO</b></font>: Give an intro of what ERDOS and Pylot is.

# Architecture of an Autonomous Vehicle

[//]: #![Pylotpipeline](./doc/source/images/pylot.png)
<img src="doc/source/images/pylot.png" alt="Pylot Pipeline" style="width: 800px;"/>

An autonomous vehicle (AV) is typically equipped with multiple instances of sensor including cameras and LiDARs that operate at different frequencies and generate approximately 1GB/s of data.

A state-of-the-art AV pipeline comprises of five modules, each implemented using several components:
1. __Perception__: Recognizes the scene by detecting and classifying objects.
2. __Localization__: Provides the location of the vehicle with decimeter-level accuracy.
3. __Prediction__: Estimates how other agents would move in the environment.
4. __Planning__: Generates trajectories for the AV to follow.
5. __Control__: Physically operates the AV.

## Tutorial Overview

In this set of tutorials, we will focus on the __*perception*__ and the __*planning*__ modules of the vehicle.

[Exercise 1](#exercise1) implements an object detector for the __*perception*__ module of an AV pipeline, and drives it around in a simulated environment.

[Exercise 2](#exercise2) <font color="red"><b>TODO</b></font>: Fill.

<a id='exercise1'></a> 
## Exercise 1: Developing a Perception component
![perception](risecamp/perception-crop.gif)

__Goal__: Get started with the [ERDOS](https://github.com/erdos-project/erdos) API by developing an object detector for an autonomous vehicle, and driving it in the [CARLA simulator](https://carla.org) using [Pylot](https://github.com/erdos-project/pylot).

ERDOS models the AV pipeline as a directed dataflow graph where developers implement the components of the pipeline as __*operators*__ that communicate with each other through typed __*streams*__. Operators can request callbacks upon receipt of messages on a stream, and send the computed results to other operators via a different stream. For example, an __*object detector*__ component runs an object detection model on incoming messages from a camera stream and sends the detected objects to the __*prediction*__ module.

### Part 1: Structure of an ERDOS Operator
An ERDOS Operator is a Python `class` which receives the input and output streams as part of its `__init__` function. 

```python
import erdos

class Operator(erdos.Operator):
    def __init__(self, input_stream_1, input_stream_2, ..., output_stream_1, output_stream_2):
        pass
```

__*[Part 2](#exercise1part2) describes how the input and output streams are connected to other components*__


Developers can request callbacks on the receipt of messages on a stream by registering the callback using the `add_callback` method.

```python
class Operator(erdos.Operator):
    def __init__(self, input_stream_1, input_stream_2, ..., output_stream_1, output_stream_2):
        input_stream_1.add_callback(self.on_message)

    def on_message(self, message):
        # Work on the message here.
```

To allow a callback to send data on an output stream, developers can pass the required output streams to the `add_callback` method. Inside the callback, developers can call the `send` method on an output stream to send the message to other operators.

```python
class Operator(erdos.Operator):
    def __init__(self, input_stream_1, input_stream_2, ..., output_stream_1, output_stream_2):
        input_stream_1.add_callback(on_message, [output_stream_1, output_stream_2])

    def on_message(self, message, output_stream_1, output_stream_2):
        # Work on the message and send the output to any of the output streams.
        result = ...
        output_stream_1.send(result)
```

We are now going to use the ERDOS API to implement an __*object detection*__ component. The following cell provides the skeleton implementation for an `ObjectDetectionOperator`. In order to get it to detect incoming objects, you have to complete the following tasks:
1. Register the `on_message` method to be invoked upon receipt of messages on the `camera_stream`. (Don't forget to provide the `detected_objects_stream` to the callback!)
2. Inside the callback, invoke the `detect_objects` method on the incoming message, and `send` the results on the `detected_objects_stream`.

In [3]:
import erdos

#from pylot.utils import load_model, infer_from_model

# The path where the checkpoint of the model is stored.
MODEL_PATH = None

class ObjectDetectionOperator(erdos.Operator):
    """Detect objects using images received on the camera stream."""
    def __init__(self, camera_stream, detected_objects_stream):
        # TODO (1): Register the `self.on_message` method here! 
        # TODO (2): Provide the `detected_objects_stream` to the method.
        self.camera_stream.add_callback()
        
    def on_message(self, message, detected_objects_stream):
        """ Receives a message with the camera frame and runs a model on the frame. """
        # TODO (3): Invoke the `detect_objects` method on the `message`, 
        #           and save results into `detected_objects`.
        
    
        # TODO (4): Send `detected_objects` to the other operators, by
        #           invoking `send` on `detected_objects_stream` and 
        #           passing the `detected_objects` as an argument.
        
        
    # Don't change the code below.
    def detect_objects(self, message):
        pass
    
    @staticmethod
    def connect(camera_stream):
        detected_objects_stream = erdos.WriteStream()
        return [detected_objects_stream]
    
# Validation code (don't change!)
from pylot.risecamp_utils import validate_problem_1
fail, err = validate_problem_1(ObjectDetectionOperator)
if fail:
    print("\x1b[31mFAILURE:\x1b[0m {}".format(err))
else:
    print("\x1b[32mSUCCESS!\x1b[0m {}".format(err))

FAILURE: self.on_message was not registered with the camera_stream


<a id='exercise1part2'></a> 
### Part 2: Connecting Operators together!

In [ ]:
a = "self.camera_stream"
help(a.split)

In [ ]:
# import inspect
# add_callback_line = list(filter(lambda s: not s.startswith('#') and s.find('add_callback') != -1,
#     map(lambda a: a.strip(), inspect.getsourcelines(ObjectDetectionOperator.__init__)[0])))

# assert len(add_callback_line) == 1, \
#     "add_callback was called twice! Make sure there is only one add_callback_invocation"

# add_callback_line = add_callback_line[0].strip() 
# arguments = add_callback_line[add_callback_line.find("(")+1:add_callback_line.find(")")].split(',', maxsplit=1)
# assert len(arguments) != 0, \
#     "add_callback was not invoked with the callback and the output stream."

# assert arguments[0] == "self.on_message", \
#     "self.on_message was not registered with the camera_stream"

# assert len(arguments) != 1, \
#     "Only a single argument was passed to the add_callback method. Did you forget the output stream?" 


# second_argument = arguments[1].strip()

# assert second_argument.startswith('[') and second_argument.endswith(']'), \
#     "The second argument to add_callback was not a list! " \
#     "Make sure you're invoking add_callback with the list of output stream(s)"

# list_args = list(map(lambda a: a.strip(), second_argument[1:-1].split(',')))
# assert len(list_args) == 1, \
#     "More than one output stream was passed to add_callback. " \
#     "Make sure to only pass the detected_objects_stream."

# assert list_args[0] == 'detected_objects_stream', \
#     "The detected_objects_stream was not passed as the second argument to add_callback"